In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig, pipeline
import torch
import pandas as pd
from peft import (
    AutoPeftModelForCausalLM,
    get_peft_model,
    LoraConfig,
    PeftConfig,
    PeftModel
)
import re

device = "cuda:0"

In [ ]:
peft_model_id = "../result/DataVortexS-10.7B-dpo-v1.11_2024-03-05_02-48-43/best"
load_tokenizer_name = "../result/DataVortexS-10.7B-dpo-v1.11_2024-03-05_02-48-43"

In [ ]:
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=False,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype="float16",
            )

models = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config)
models = PeftModel.from_pretrained(models, peft_model_id)
models = models.merge_and_unload() # 7481 -> 8047

tokenizers = AutoTokenizer.from_pretrained(load_tokenizer_name)

In [ ]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test_split.csv")

print(tokenizers.pad_token)
print(tokenizers.eos_token)

test.head(5)

In [ ]:
input_text = '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?'

with open("../template/datavortex.txt", "r", encoding="utf-8") as file:
    content = file.read()

answer_start_index = content.find("<answer>")
content = content[:answer_start_index]
content = content.replace("<question>", input_text)

input_ids = tokenizers.encode(
            content, padding=False, max_length=256, return_tensors="pt", add_special_tokens=False
        )
print(content, "\n--------------\n")
print(content.replace("\n", "\\n"), "\n--------------\n")
print(input_ids)

In [ ]:
output_sequences = models.generate(
        input_ids=input_ids.to("cuda"),
        max_length=512,
        temperature=0.2,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

print(tokenizers.decode(output_sequences[0]))

---

# similarity

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

In [ ]:
k = model.encode(["방청 페인트는 다양한 종류로 분류됩니다 . 그 중에서 가장 흔 히 사용되는 것은 광 명단 페인 트, 방청산화철 페인 트, 알루미늄 페인 트, 역청질 페인 트, 워시 프라이머 , 크롬 산아연 페인 트 등이 있습니다 . 이러한 다양한 종류 의 방청 페인트가 각자의 특성과 용도에 맞 게 사용됩니다 ."])
t = model.encode(["방청 페인트는 다양한 종류로 분류됩니다. 그 중에서 가장 흔히 사용되는 것은 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬 산아연 페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트가 각자의 특성과 용도에 맞게 사용됩니다."])
cosine_similarity(k, t)

---

# start token이 잘못되어 보정

In [ ]:
import json
import pandas as pd
import re
from sentence_transformers import SentenceTransformer

In [ ]:
def submission(CFG, preds):
    model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
    nl = pd.read_csv(f"../{CFG['DATA_PATH']}/{CFG['TEST_DATA']}")
    submit = pd.read_csv(f"../{CFG['DATA_PATH']}/{CFG['SUBMISSION_DATA']}")
    submission_name = "DataVortexS-10.7B-dpo-v1.11_2024-03-04_02-11-13"
    nl["답변"] = preds
    nl.to_csv(f'../{CFG["SAVE_PATH"]}/{submission_name}/NL_{submission_name}.csv', index=False)
    if len(nl) != len(submit):
        nl = (
            nl.groupby("id")["답변"]
            .apply(lambda x: " ".join(x.astype(str)))
            .reset_index()
        )
        preds = nl["답변"]
        nl.to_csv(f'../{CFG["SAVE_PATH"]}/{submission_name}/NL_merge_{submission_name}.csv', index=False)
    pred_embeddings = model.encode(preds)
    print("Shape of Prediction Embeddings: ", pred_embeddings.shape)
    submit.iloc[:, 1:] = pred_embeddings
    submit.to_csv(f'../{CFG["SAVE_PATH"]}/{submission_name}/{submission_name}.csv', index=False)

In [ ]:
with open(f'../config/datavortex_0304.json', 'r', encoding='utf-8') as file:
    CFG = json.load(file)
    
infer_csv = pd.read_csv("../result/NL_DataVortexS-10.7B-dpo-v1.11_2024-03-04_02-11-13.csv")

In [ ]:
preds = []
start_token = "### Assistant: "
for full_text in infer_csv['답변']:
    answer_start = full_text.find(start_token) + len(start_token)
    answer_only = full_text[answer_start:].strip()
    answer_only = answer_only.replace("\n", "")
    answer_only = re.sub(r'\s+', ' ', answer_only)
    answer_only = answer_only.replace("<|im_end|>", "")
    answer_only = answer_only.replace("</s>", "")
    preds.append(answer_only)

In [ ]:
submission(CFG, preds)